In [7]:
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
! hostname

n405d3rg96


In [9]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [12]:
df = pd.read_csv('./data/merged.csv')
# df = pd.read_csv('../data/merged_labeled_daily.csv')

In [13]:
df = df.drop(columns=df.columns[:2])
df['measure_date'] = pd.to_datetime(df['measure_date'])
df['year'] = df['measure_date'].dt.year
df['month'] = df['measure_date'].dt.month
df['day'] = df['measure_date'].dt.day
df['weekday'] = df['measure_date'].dt.weekday
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)

In [14]:
df.head()

,station_code,measure_date,HS,no_snow,anomaly,year,month,day,weekday,month_sin,month_cos,day_sin,day_cos
0,AMD2,1997-10-14,12.0,False,False,1997,10,14,1,-0.866025,0.5,0.299363,-0.954139
1,AMD2,1997-10-15,31.0,False,False,1997,10,15,2,-0.866025,0.5,0.101168,-0.994869
2,AMD2,1997-10-16,9.0,False,False,1997,10,16,3,-0.866025,0.5,-0.101168,-0.994869
3,AMD2,1997-10-17,16.0,False,False,1997,10,17,4,-0.866025,0.5,-0.299363,-0.954139
4,AMD2,1997-10-18,-4.0,True,False,1997,10,18,5,-0.866025,0.5,-0.485302,-0.874347


In [15]:
num_train_stations = 15
stations = df['station_code'].unique()

train_val_end_index = df[df['station_code'] == stations[num_train_stations - 1]].index[-1] + 1

num_train_samples = int(0.7 * train_val_end_index)
num_val_samples = train_val_end_index - num_train_samples
num_test_samples = len(df) - train_val_end_index

print("Train length: %s" % num_train_samples)
print("Validation length: %s" % num_val_samples)
print("Test length: %s" % num_test_samples)

total = num_train_samples + num_val_samples + num_test_samples
print(total, len(df), total == len(df))

Train length: 88601
Validation length: 37972
Test length: 42749
169322 169322 True


In [16]:
features = df[['HS', 'day_sin', 'day_cos', 'month_sin', 'month_cos']].values
target = df[['no_snow']].values

In [17]:
mean = features[:num_train_samples].mean(axis=0)
features -= mean
std = features[:num_train_samples].std(axis=0)
features /= std
print(features)

[[-0.65719526  0.41694945 -1.33643084 -1.22485271  0.69111721]
 [-0.46035473  0.13911521 -1.39456923 -1.22485271  0.69111721]
 [-0.68827535 -0.14452516 -1.39456923 -1.22485271  0.69111721]
 ...
 [ 0.2783153   1.10582347 -0.84821005  1.23528619  0.69111721]
 [ 0.23791118  0.91040464 -1.05754238  1.23528619  0.69111721]
 [ 0.26277525  0.67760299 -1.22253424  1.23528619  0.69111721]]


In [18]:
print(target[:3])
print(features[:3])

[[False]
 [False]
 [False]]
[[-0.65719526  0.41694945 -1.33643084 -1.22485271  0.69111721]
 [-0.46035473  0.13911521 -1.39456923 -1.22485271  0.69111721]
 [-0.68827535 -0.14452516 -1.39456923 -1.22485271  0.69111721]]


In [19]:
def sequence_debugging():
    int_sequence = np.arange(30)

    train_end_index = int(0.5 * len(int_sequence))
    val_end_index = int(0.75 * len(int_sequence))

    sequence_length = 3
    target_start_idx = sequence_length - 1

    train_dataset = keras.utils.timeseries_dataset_from_array(
        data=int_sequence,
        targets=int_sequence[target_start_idx:],
        sequence_length=sequence_length,
        start_index=0,
        end_index=train_end_index
    )

    val_dataset = keras.utils.timeseries_dataset_from_array(
        data=int_sequence,
        targets=int_sequence[target_start_idx:],
        sequence_length=sequence_length,
        start_index=train_end_index,
        end_index=val_end_index
    )

    test_dataset = keras.utils.timeseries_dataset_from_array(
        data=int_sequence,
        targets=int_sequence[target_start_idx:],
        sequence_length=sequence_length,
        start_index=val_end_index
    )

    print("Train:")
    for inputs, targets in train_dataset:
        for i in range(inputs.shape[0]):
            print([int(x) for x in inputs[i]], int(targets[i]))

            
    print("\nVal:")
    for inputs, targets in val_dataset:
        for i in range(inputs.shape[0]):
            print([int(x) for x in inputs[i]], int(targets[i]))

    print("\nTest:")
    for inputs, targets in test_dataset:
        for i in range(inputs.shape[0]):
            print([int(x) for x in inputs[i]], int(targets[i]))

In [20]:
import numpy as np
from tensorflow import keras

sequence_length = 30
sequence_stride = 1
sampling_rate = 1
sequence_batch_size = 128
# batch_size = None
shuffle_train_val = True
shuffle_test = False
start_index = 0

target_start_idx = sequence_length - 1

def create_dataset(start_index, end_index, shuffle=False):
    return keras.utils.timeseries_dataset_from_array(
        data=features,
        targets=target[target_start_idx:],
        sequence_length=sequence_length,
        sequence_stride=sequence_stride,
        sampling_rate=1,
        batch_size=sequence_batch_size,
        shuffle=shuffle,
        start_index=start_index,
        end_index=end_index
    )

train_dataset = create_dataset(start_index=0, end_index=num_train_samples, shuffle=shuffle_train_val)
val_dataset = create_dataset(start_index=num_train_samples, end_index=num_train_samples + num_val_samples, shuffle=shuffle_train_val)
test_dataset = create_dataset(start_index=num_train_samples + num_val_samples, end_index=(len(df) - 1), shuffle=shuffle_test)

In [21]:
import re

class MonteCarloDropout(keras.layers.Dropout):
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)
    
def create_network(architecture, input_shape, dropout_rate=0.5, summary=True):
    arch_split = architecture.split('-')
    dense = True
    bidirectional = False
    layers = []

    digits_pattern = re.compile(r"\d+")
    
    if 'l' in arch_split[0]:
        rnn_layer = 'LSTM'
    elif 'g' in arch_split[0]:
        rnn_layer = 'GRU'
    else:
        raise Exception('rnn_layers must be one of [LSTM, GRU]')

    if 'b' in arch_split[0]:
        bidirectional = True
        
    for i, layer in enumerate(reversed(arch_split)):
        no_units = int(digits_pattern.findall(layer)[0])

        if dense:
            activation = 'sigmoid' if i == 0 else 'relu'
            layers.append(keras.layers.Dense(no_units, activation=activation))
        else:
            args = {
                'units': no_units,
            }

            if '(d)' not in arch_split[-i]:
                args['return_sequences'] = True

            if i == len(arch_split) - 1:
                args['input_shape'] = input_shape
                
            current_layer = keras.layers.LSTM(**args) if rnn_layer == 'LSTM' else keras.layers.GRU(**args)
            
            if bidirectional:
                current_layer = keras.layers.Bidirectional(current_layer)
            
            layers.extend([
                MonteCarloDropout(dropout_rate),
                current_layer
            ])
            
        if '(d)' in layer:
            dense = False
            
    layers.reverse()
    
    if summary:
        for layer in layers:
            layer_type = str(type(layer))

            if 'Dense' in str(type(layer)):
                print(type(layer), layer.units, layer.activation)
            elif 'Bidirectional' in str(type(layer)):
                print(type(layer), layer.layer, layer.layer.units, layer.layer.activation, layer.layer.return_sequences)
            elif 'LSTM' in str(type(layer)):
                print(type(layer), layer.units, layer.activation, layer.return_sequences)
            elif 'GRU' in str(type(layer)):
                print(type(layer), layer.units, layer.activation, layer.return_sequences)
            elif 'Dropout' in layer_type:
                print(type(layer), layer.rate)

    return layers

# arch = "128(gb)-64-8(d)-1"
# input_shape = (sequence_length, features.shape[-1])
# network = create_network(arch, input_shape)


In [22]:
arch = "128(l)-64-8(d)-1"
input_shape = (sequence_length, features.shape[-1])
epochs = 10
model_batch_size = 64
checkpoint_name = 'snow_height_anomaly_multi_vars_gru.keras'
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_name = f"{current_time}_arch-{arch}_epochs-{epochs}_batch-{model_batch_size}_seq-{sequence_length}_num-train-stations-{num_train_stations}"
log_dir = f"./logs/{model_name}"

network = create_network(arch, input_shape)
model = keras.Sequential(network)
model.compile(
    optimizer='adam',
    metrics='accuracy',
    loss='binary_crossentropy'
)

history = model.fit(
    train_dataset,
    epochs=epochs,
    batch_size=model_batch_size,
    validation_data=val_dataset,
    # callbacks=[
        # keras.callbacks.ModelCheckpoint(checkpoint_name, save_best_only=True),
        # keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) 
    # ]
)

<class 'keras.layers.rnn.lstm.LSTM'> 128 <function tanh at 0x7fbfb16e9040> True
<class '__main__.MonteCarloDropout'> 0.5
<class 'keras.layers.rnn.lstm.LSTM'> 64 <function tanh at 0x7fbfb16e9040> False
<class '__main__.MonteCarloDropout'> 0.5
<class 'keras.layers.core.dense.Dense'> 8 <function relu at 0x7fbfb16e4ca0>
<class 'keras.layers.core.dense.Dense'> 1 <function sigmoid at 0x7fbfb16e91f0>
Epoch 1/10


KeyboardInterrupt: 

In [23]:
def plot_keras_history(history):
    # Plot training & validation loss values
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

plot_keras_history(history)

NameError: name 'history' is not defined

In [26]:
model.evaluate(test_dataset)

334/334 [==============================] - 5s 15ms/step - loss: 0.2361 - accuracy: 0.9214


[0.23606866598129272, 0.9214401245117188]

In [24]:
def plot_data(dfs, y='HS', target='no_snow', predictions=None):
    if not isinstance(dfs, list):
        dfs = [dfs]

    if not isinstance(predictions, list):
        predictions = [predictions] if predictions else []

    if not isinstance(y, list):
        y = [y] * len(dfs)

    if not isinstance(target, list):
        target = [target] * len(dfs)

    if len(predictions):
        rows = len(dfs) * 2
    else:
        rows = len(dfs)

    fig, axes = plt.subplots(rows, 1, figsize=(20, 5 * rows))

    if not isinstance(axes, np.ndarray):
        axes = [axes]

    for i, df in enumerate(dfs):
        plot_df = df.copy()
        plot_df.index = pd.to_datetime(plot_df['measure_date'])
        station_code = plot_df['station_code'].iloc[0]

        # Plot original data
        ax = axes[2 * i] if len(predictions) else axes[i]
        ax.set_xlabel('Date')
        ax.set_ylabel(y[i])
        ax.set_title(f'Station: {station_code} | Original Data with Anomalies Highlighted')
        ax.plot(plot_df.index, plot_df[y[i]], label='Data', marker='o', linestyle='-', ms=4)
        ax.scatter(plot_df[plot_df[target[i]]].index, plot_df[plot_df[target[i]]][y[i]], color='red', label='Anomalies', zorder=5, s=20)
        ax.legend()

        # Plot predictions
        if len(predictions):
            ax = axes[2 * i + 1]
            ax.set_xlabel('Date')
            ax.set_ylabel(y[i])
            ax.set_title(f'Station: {station_code} | Predicted Data with Anomalies Highlighted')
            ax.plot(plot_df.index, plot_df[y[i]], label='Data', marker='o', linestyle='-', ms=4)
            ax.scatter(plot_df[predictions[i]].index, plot_df[predictions[i]][y[i]], color='green', label='Predicted Anomalies', zorder=5, s=20)
            ax.legend()

    plt.tight_layout()
    plt.show()

In [28]:
test_stations = df[train_val_end_index:]['station_code'].unique()
test_stations = [(df[df['station_code'] == station].index.values[[0, -1]], df[df['station_code'] == station][sequence_length:]) for station in test_stations]
predictions = [model.predict(create_dataset(station[0][0], station[0][1])).reshape((-1,)) > 0.5 for station in test_stations]
# 
plot_data(
    [station[1] for station in test_stations],
    predictions=predictions
)

In [89]:
def predict_with_uncertainty(model, x, n_iter=100):
    preds = np.array([model.predict(x) for _ in range(n_iter)])
    uncertainty = np.std(preds, axis=0)
    scaled_uncertainty = (uncertainty - uncertainty.min()) / (uncertainty.max() - uncertainty.min())
    return scaled_uncertainty.mean()

uncertainties = [predict_with_uncertainty(model, create_dataset(station[0][0], station[0][1]), n_iter=20) for station in test_stations]
accuracy = [model.evaluate(create_dataset(station[0][0], station[0][1]))[1] for station in test_stations]

print(np.corrcoef(uncertainties, accuracy))


87/87 [==============================] - 1s 7ms/step - loss: 0.2320 - accuracy: 0.9238
[[ 1.         -0.57381575]
 [-0.57381575  1.        ]]


In [90]:
print(uncertainties)
print(accuracy)
print(np.corrcoef(uncertainties, accuracy)[0, 1])


[0.04935129, 0.049654353, 0.08818548, 0.032795604, 0.041576687]
[0.9533082246780396, 0.9526672959327698, 0.8761699199676514, 0.9086413979530334, 0.9238069653511047]
-0.5738157518119447


In [ ]:
# 1. Pick random sample out of dataset


# 3. Run a model on it

# 4. Predict data on all other models

# 5. Assign uncertainty scores to all other models

# 6. Pick most uncertain sample

# 7. Repeat

# 4. Assign uncertainty scores to all other models

#